In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.callbacks import *

In [ ]:
torch.cuda.is_available()

In [ ]:
np.random.seed(42)

In [ ]:
tfms = get_transforms(do_flip=True,flip_vert=False,max_rotate=10.0,max_zoom=1.2,max_lighting=0.5,max_warp=0.2,p_affine=0.75,p_lighting=0.75)


In [ ]:
mixup=0.2

In [ ]:
loss_func = LabelSmoothingCrossEntropy()

In [ ]:
# class FocalLoss(nn.Module):
#     def __init__(self, alpha=1., gamma=1.):
#         super().__init__()
#         self.alpha = alpha
#         self.gamma = gamma

#     def forward(self, inputs, targets, **kwargs):
#         CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
#         pt = torch.exp(-CE_loss)
#         F_loss = self.alpha * ((1-pt)**self.gamma) * CE_loss
#         return F_loss.mean()
# loss_func = FocalLoss(gamma=2.)

In [ ]:
alpha=0.99
mom=0.9
eps=1e-6

In [ ]:
opt_func = partial(optim.Adam, betas=(mom,alpha), eps=eps)

In [ ]:
src = (ImageList.from_folder(path='train').split_by_rand_pct(0.2).label_from_folder())

In [ ]:
# from pathlib import Path
# path = Path("round1/")

# print(path)
# verify_images(path, delete=True)

In [ ]:
src.add_test_folder('/home/ubuntu/snakes_breed/round1')

In [ ]:
data = src.transform(tfms,size=(640)).databunch(bs=60).normalize()

In [ ]:
data.test_ds.items[0]

In [ ]:
learn = cnn_learner(data, models.densenet121,pretrained=False, metrics=[error_rate, accuracy, FBeta(beta=1,average='macro')],loss_func = loss_func,opt_func=opt_func,wd=1e-2)

In [ ]:
learn= learn.mixup(alpha=mixup)

In [ ]:
learn.bn_wd=False

In [ ]:
learn.true_wd=True

In [ ]:
learn.load('exp-11-round-1')

In [ ]:
learn.model.eval()

In [ ]:
preds,output=learn.TTA(ds_type=DatasetType.Test)

In [ ]:
preds[0]

In [ ]:
probs = np.exp(preds) / np.exp(preds).sum(1)[:,None]

In [ ]:
probs[0]

In [ ]:
df_test = pd.read_csv('test_file.csv',low_memory=False)

In [ ]:
df_test.head()

In [ ]:
data.classes[np.argmax(probs[0])]

In [ ]:
df_classes = pd.read_csv('class.csv',low_memory=False)

In [ ]:
df_classes.head()

In [ ]:
data_dict = df_classes.set_index('class_idx')['original_class'].to_dict()

In [ ]:
data_dict

In [ ]:
data.test_ds.items

In [ ]:
print(os.path.basename(data.test_ds.items[0]))

In [ ]:
df_test.head()

In [ ]:
probs_np = probs.numpy()

In [ ]:
df_testfile_mapping = pd.DataFrame(columns=['filename','map'])

In [ ]:
df_testfile_mapping['filename']=df_test['filename']

In [ ]:
for i,row in df_testfile_mapping.iterrows():
    row['map']=i
    

In [ ]:
df_testfile_mapping.head()

In [ ]:
data_dict_filename = df_testfile_mapping.set_index('filename')['map'].to_dict()

In [ ]:
i = 0
for test_file in data.test_ds.items:
    filename = (os.path.basename(test_file))
    map_val = int(data_dict_filename[filename])
    for c in range(0,45):
        df_test.loc[map_val,data_dict[int(data.classes[c].split("-")[1])]]=probs_np[i][c]
    i += 1
    print(i)

In [ ]:
df_test.to_csv('submission-2.csv',index=False)

In [ ]:
for i,row in df_test.iterrows():
    sum_temp=row[1:46].sum()
    low_limit = 1-1e-6
    high_limit = 1+1e-6
    print(sum_temp)
    assert sum_temp>= low_limit and sum_temp <= high_limit


In [ ]:
#submission










